# F1 Elo Prototyping

Notebook for exploring ideas before binning or porting to scripts.

## Environment setup

In [1]:
import yaml

import pandas as pd
import plotly.express as px

In [2]:
# Move project root folder
%cd ..

/Users/mitchell/projects/personal/f1-rating-system


/Users/mitchell/Library/Caches/pypoetry/virtualenvs/f1-rating-system-bj5GO79q-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Data importing

In [3]:
with open("params.yaml") as conf_file:
    CONFIG = yaml.safe_load(conf_file)

mod_df = pd.read_csv(CONFIG["data"]["modelled_path"])
constructors_df = pd.read_csv(CONFIG["data"]["constructors_csv"])[["constructorId", "name"]].rename(columns={"name": "constructorName"})
drivers_df = pd.read_csv(CONFIG["data"]["drivers_csv"])[["driverId", "driverRef"]]

mod_df = pd.read_csv(CONFIG["data"]["modelled_path"])

# replace driver ID with driver name for visualisation
drivers_df = pd.read_csv(CONFIG["data"]["drivers_csv"])[["driverId", "forename", "surname"]]
drivers_df["driverName"] = drivers_df[["forename", "surname"]].apply(lambda row: " ".join(row), axis=1)
vis_df = mod_df.merge(drivers_df[["driverId", "driverName"]], on=["driverId"], how="left")

con_df = pd.read_csv(CONFIG["data"]["constructors_csv"])[["constructorId", "name"]]
con_df = con_df.rename(columns={"name": "constructorName"})
vis_df = vis_df.merge(con_df, on="constructorId", how="left")

df24 = pd.read_csv(CONFIG["data"]["2024_path"])

In [111]:
# Capturing constructor periods to begin tracking buyouts and rebrands to integrate into app
import numpy as np

results_df = pd.read_csv(CONFIG["data"]["results_csv"])
races_df = pd.read_csv(CONFIG["data"]["races_csv"])
con_df = pd.read_csv(CONFIG["data"]["constructors_csv"])[["constructorId", "name"]].rename(columns={"name": "constructorName"})

period_df = results_df[["raceId", "constructorId"]].drop_duplicates().reset_index(drop=True)
period_df = period_df.merge(races_df[["raceId", "year"]], on="raceId", how="left")
period_df = period_df.merge(con_df[["constructorId", "constructorName"]], on="constructorId", how="left")

con_yr_df = period_df[["constructorId", "constructorName", "year"]].drop_duplicates()
con_yr_df["n_years"] = con_yr_df.groupby("constructorId")["year"].transform("count")
con_yr_df = con_yr_df.sort_values(["constructorId", "n_years", "year"]).reset_index(drop=True)

con_yr_df.to_csv("constructor_year.csv", index=False)

In [17]:
df24.sort_values(["round", "driverId"]).loc[df24["constructorName"] == "Ferrari", ["round", "driverName", "startConScore", "constructorScore", "status", "conScoreChange"]]

round       driverName  startConScore  constructorScore  \
2        1     Carlos Sainz    2442.497103       2479.971229   
3        1  Charles Leclerc    2479.971229       2479.971229   
22       2  Charles Leclerc    2479.971229       2507.323572   
26       2   Oliver Bearman    2507.323572       2507.323572   
40       3     Carlos Sainz    2507.323572       2562.080425   
41       3  Charles Leclerc    2562.080425       2562.080425   
61       4     Carlos Sainz    2562.080425       2591.320449   
62       4  Charles Leclerc    2591.320449       2591.320449   
83       5     Carlos Sainz    2607.622979       2607.622979   
82       5  Charles Leclerc    2591.320449       2607.622979   
103      6     Carlos Sainz    2628.806911       2628.806911   
101      6  Charles Leclerc    2607.622979       2628.806911   
123      7     Carlos Sainz    2648.169158       2648.169158   
121      7  Charles Leclerc    2628.806911       2648.169158   
141      8     Carlos Sainz    2686.908269       2686.908269   
139      8  Charles Leclerc    2648.169158       2686.908269   
176      9     Carlos Sainz    2540.968259       2540.968259   
175      9  Charles Leclerc    2686.908269       2540.968259   
184     10     Carlos Sainz    2548.748550       2548.748550   
183     10  Charles Leclerc    2540.968259       2548.748550   
201     11     Carlos Sainz    2548.748550       2540.885201   
209     11  Charles Leclerc    2540.885201       2540.885201   
223     12     Carlos Sainz    2540.885201       2507.423752   
232     12  Charles Leclerc    2507.423752       2507.423752   
244     13     Carlos Sainz    2522.638445       2522.638445   
242     13  Charles Leclerc    2507.423752       2522.638445   

                     status  conScoreChange  
2                  finished       37.474126  
3                  finished        0.000000  
22                 finished       27.352343  
26                 finished        0.000000  
40                 finished       54.756853  
41                 finished        0.000000  
61                 finished       29.240023  
62                 finished        0.000000  
83                 finished        0.000000  
82                 finished       16.302530  
103                finished        0.000000  
101                finished       21.183932  
123                finished        0.000000  
121                finished       19.362247  
141                finished        0.000000  
139                finished       38.739110  
176       driver retirement        0.000000  
175  constructor retirement     -145.940010  
184                finished        0.000000  
183                finished        7.780291  
201                finished       -7.863348  
209                finished        0.000000  
223                finished      -33.461449  
232                finished        0.000000  
244                finished        0.000000  
242                finished       15.214693

In [8]:
df24[["constructorId", "constructorName", "conScoreChange"]].drop_duplicates().groupby(["constructorId", "constructorName"])["conScoreChange"].sum().reset_index()

constructorId constructorName  conScoreChange
0              1         McLaren      378.834375
1              3        Williams      -62.961433
2              6         Ferrari     1169.574110
3              9        Red Bull     -122.204500
4             15          Sauber      473.670866
5            117    Aston Martin     -108.717601
6            131        Mercedes      -13.642658
7            210    Haas F1 Team      347.348579
8            214  Alpine F1 Team     -224.715036
9            215      RB F1 Team      125.309234

In [40]:
vis_df["outperformance"] = vis_df["actual"] - vis_df["expected"]
out_df = vis_df.groupby("driverId")["outperformance"].mean().reset_index()
count_df = vis_df.groupby("driverId").size().reset_index().rename(columns={0: "races"})
max_df = vis_df.groupby("driverId")["driverScore"].max().reset_index().rename(columns={"driverScore": "maxScore"})

agg_df = vis_df.groupby(["driverId", "driverName"])["driverScore"].mean().reset_index().rename(columns={"driverScore": "meanScore"})
agg_df = agg_df.merge(out_df, on="driverId", how="left").merge(count_df, on="driverId", how="left").merge(max_df, on="driverId", how="left")
agg_df = agg_df[["driverId", "driverName", "races", "meanScore", "outperformance", "maxScore"]]

goat_df = agg_df.sort_values("meanScore", ascending=False).head(20).reset_index(drop=True)
goat_hist_df = vis_df[vis_df["driverId"].isin(goat_df["driverId"])]

goat_hist_df.head()

year  round        date  constructorId  driverId  mapPosition  mapPoints  \
12   1950      1  1950-05-13             51       579           12        0.0   
23   1950      2  1950-05-21             51       579            1       25.0   
90   1950      4  1950-06-04             51       579           12        0.0   
97   1950      5  1950-06-18             51       579            1       25.0   
111  1950      6  1950-07-02             51       579            1       25.0   

                     status  constructorScore  driverScore  expected  actual  \
12   constructor retirement       1523.560845  1500.000000  0.000000       0   
23                 finished       1552.595610  1529.487883  9.021285      17   
90   constructor retirement       1562.455044  1529.487883  0.000000       0   
97                 finished       1582.869833  1554.526005  5.413386       9   
111                finished       1595.084940  1575.674296  5.307190       8   

      driverName constructorName  outperformance  
12   Juan Fangio      Alfa Romeo        0.000000  
23   Juan Fangio      Alfa Romeo        7.978715  
90   Juan Fangio      Alfa Romeo        0.000000  
97   Juan Fangio      Alfa Romeo        3.586614  
111  Juan Fangio      Alfa Romeo        2.692810

In [32]:

vis_df["outperformance"] = vis_df["actual"] - vis_df["expected"]
vis_df.groupby(["driverId", "driverName"])["outperformance"].mean().reset_index().sort_values("outperformance", ascending=False).reset_index(drop=True).head(20)

driverId      driverName  outperformance
0        591    George Amick       10.775499
1        674  Larry Crockett        9.045969
2        794   Joie Chitwood        8.000764
3        743   Eric Thompson        6.669871
4        701    Bill Holland        5.750382
5        766     Lee Wallard        5.709228
6        457         Ben Pon        5.683761
7        673    Mike Nazaruk        5.668481
8        770     Cecil Green        4.750382
9        656       Art Cross        4.733178
10       323  Dieter Quester        4.551377
11       742    Dennis Poore        4.397732
12       657   Bill Vukovich        4.284304
13       518    Duane Carter        4.115586
14       555      Paul Russo        3.775463
15       658    Jack McGrath        3.603653
16       525     Jimmy Bryan        3.437360
17       628    Bob Sweikert        3.270471
18       695    Oscar Gálvez        3.229014
19       768      Mauri Rose        3.069682

In [4]:
dri_df = pd.read_csv(CONFIG["data"]["2024_path"])

In [26]:
# constructor colours
con_df = pd.DataFrame({
    "constructorId": [9, 1, 6, 131, 117, 214, 210, 215, 3, 15],
    "colour": ['#3671C6', '#FF8100', '#E80020', '#25F4D2', '#219971', '#0293CC', '#B6BABD', '#6692FF', '#63C4FF', '#51E251']
})

import json

con_dict = {id: col for _, (id, col) in con_df.iterrows()}


json.dumps(con_dict)

'{"9": "#3671C6", "1": "#FF8100", "6": "#E80020", "131": "#25F4D2", "117": "#219971", "214": "#0293CC", "210": "#B6BABD", "215": "#6692FF", "3": "#63C4FF", "15": "#51E251"}'

## Data visualisation

In [21]:
con_df = mod_df.groupby(["year", "date", "constructorId"])["constructorScore"].max().reset_index()
con_df = con_df.merge(constructors_df, on="constructorId", how="left")
cur_con = set(con_df.loc[con_df["year"] == 2024, "constructorId"])
cur_df = con_df[con_df["constructorId"].isin(cur_con) & (con_df["year"] > 2010)]

px.line(cur_df, x="date", y="constructorScore", color="constructorName")

In [22]:
cur_df.loc[(cur_df["date"] == cur_df["date"].max()), ["constructorName", "constructorScore"]].sort_values("constructorScore", ascending=False).drop_duplicates()

constructorName  constructorScore
12871        Red Bull       1796.971819
12870         Ferrari       1726.404103
12874        Mercedes       1701.023825
12868         McLaren       1685.843528
12873    Aston Martin       1572.046843
12872          Sauber       1548.079025
12877      RB F1 Team       1518.244900
12876  Alpine F1 Team       1490.258123
12875    Haas F1 Team       1485.018449
12869        Williams       1458.211512

In [24]:
dri_df = mod_df.groupby(["year", "date", "driverId"])["driverScore"].max().reset_index()
dri_df = dri_df.merge(drivers_df, on="driverId", how="left")
cur_dri = set(dri_df.loc[dri_df["year"] == 2024, "driverId"])
cur_dri_df = dri_df[dri_df["driverId"].isin(cur_dri) & (dri_df["year"] > 2010)]

px.line(cur_dri_df, x="date", y="driverScore", color="driverRef")

In [25]:
cur_dri_df.loc[(cur_dri_df["date"] == cur_dri_df["date"].max()), ["driverRef", "driverScore"]].sort_values("driverScore", ascending=False).drop_duplicates()

driverRef  driverScore
26355   max_verstappen  1961.950539
26361           norris  1815.509350
26349           alonso  1751.968031
26360          leclerc  1739.437965
26348         hamilton  1688.052404
26356            sainz  1662.909169
26366          piastri  1641.439192
26357             ocon  1630.923083
26362          russell  1616.181912
26359            gasly  1607.877850
26351            perez  1585.516069
26350       hulkenberg  1573.982613
26363            albon  1569.255114
26364          tsunoda  1561.991618
26352        ricciardo  1556.220201
26358           stroll  1541.984222
26353           bottas  1518.640260
26365             zhou  1480.926169
26354  kevin_magnussen  1480.541524
26367         sargeant  1421.054703

# Modelling optimisation

In [ ]:
# runtime improvement per model run: 32.4s -> 2.89s